In [1]:
#dowloading the cifar dataset from inernet

In [11]:
from urllib.request import urlretrieve
from os.path import isfile,isdir
from tqdm import tqdm
import  tarfile 

ModuleNotFoundError: No module named 'tqdm'

In [23]:
cifar_10_batches_folder_path = 'cifar-10-batches-py'

In [ ]:
'''checking if file is already downloaded or not'''

if not isfile('cifar-10-python.tar.gz'):
    with DownloadProgress(unit = 'B',unit_scale = True , miniters = 1, desc = 'cifar 10 dataset') as pbar:
        urlretrieve(            'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz',
            'cifar-10-python.tar.gz',
                   pbar.hook)
'''check whether folder present or not'''
if not isdir(cifar_10_batches_folder):
    with tarfile.open(cifar-10-python.tar.gz) as tar:
        tar.extractall()
        tar.close()

In [12]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
%

UsageError: Line magic function `%` not found.


In [8]:
def load_lable_name():
    return ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [7]:
def load_cifar10_batch(cifar_10_batches_folder_path,batch_id):
    with open(cifar_10_batches_folder + '/data_batch_' + str(batch_id), mode = 'rb') as file:
        batch = pickle.load(file, encoding = 'latin1')
    
    features = batch['data'].reshape((len(batch['data']),3,32,32)).transpose(0,2,3,1)
    labels = batch['labels']
    return features,labels      

Understanding the data before going further

In [14]:
features,labels = load_cifar10_batch(cifar_10_batches_folder,1)
len(features)
features[700].max()
plt.imshow(features[700])
print('the image name is {}'.format(load_lable_name()[labels[700]]))

In [15]:
features,labels = load_cifar10_batch(cifar_10_batches_folder,1)
int(len(features)*0.1)

1000

In [1]:
#Normalizing the input data

def normalize(x):
    min_val = np.min(x)
    max_val = np.max(x)
    x = (x - min_val)/(max_val-min_val)
    return x

In [25]:
# one_hot encoding the labels
def one_hot_encode(x):
    '''x is the lables'''
    encoded = np.zeros((len(x),10)) 
    for idx,value in enumerate(x):
        encoded[idx][value] = 1
    return encoded
        

In [34]:
def _preprocess_and_save(normalize, one_hot_encode ,features, labels, filename):
    features = normalize(features)
    labels   = one_hot_encode(labels)
    
    pickle.dump((features,labels),open(filename, 'wb'))

In [47]:
def preprocess_and_save(cifar_10_batches_folder_path, normalize, one_hot_encode):
        n_batches = 5
        valid_features = []
        valid_labels   = []
        
        for batch_i in range(1, n_batches+1):
            features,labels = load_cifar10_batch(cifar_10_batches_folder_path, batch_i)
            
            index_validation = int(len(features) * 0.1)
            '''
            preprocess the 90% of the data and normalize and encode it
            '''
            _preprocess_and_save(normalize, one_hot_encode ,features[:-index_validation], labels[:-index_validation], 
                                 'preprocess_batch_'+str(batch_i) + '.p')
            
            '''take 10 % datain to validation set'''
            
            valid_features.extend(features[-index_validation:])
            valid_labels.extend(labels[-index_validation:])
        
        # preprocess the stackesd validation set
        _preprocess_and_save(normalize, one_hot_encode ,np.array(valid_features), np.array(valid_labels),
                             'preprocess_validation.p')
        
        # preprocess and save the test batch
        with open("C:/Pyhton/Git_Ds_Projects/cnn/cifar_10_batches_py/test_batch", mode = 'rb') as file:
            batch = pickle.load(file, encoding = 'latin1')
        
        features = batch['data'].reshape(len(batch['data']),3,32,32).transpose(0,2,3,1)
        labels = batch['labels']
        
        # preprocess 
        _preprocess_and_save(normalize, one_hot_encode ,features, labels, 'preprocess_training.p')

In [48]:
preprocess_and_save(cifar_10_batches_folder_path,normalize,one_hot_encode)

# Check point

In [51]:
import pickle 


In [53]:
valid_features,valid_labels = pickle.load(open('preprocess_validation.p', mode = 'rb'))

In [56]:
valid_features.shape

(5000, 32, 32, 3)

# Tensoflow

In [7]:
import tensorflow as tf

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [8]:
# this will remove old biases and weights
tf.reset_default_graph()

In [9]:
#i/ps

x = tf.placeholder(tf.float32, shape = (None,32,32,3), name = 'input_x')
y = tf.placeholder(tf.float32, shape = (None,10), name = 'output_y')
keep_prob = tf.placeholder(tf.float32 , name = 'keep_prob')

# convultion nets

In [16]:
def conv_net(x, keep_prob):
    #filters
    conv1_filter = tf.Variable(tf.truncated_normal(shape = [3,3,3,64] , mean = 0 , stddev = 0.08))
    conv2_filter = tf.Variable(tf.truncated_normal(shape = [3,3,64,128],mean = 0 , stddev = 0.08))
    conv3_filter = tf.Variable(tf.truncated_normal(shape = [5,5,128,256], mean = 0, stddev = 0.08))
    conv4_filter = tf.Variable(tf.truncated_normal(shape = [5,5,256,512],mean = 0, stddev = 0.08))
    
    # 1,2 layers
    
    conv1 = tf.nn.conv2d(x , conv1_filter , strides = [1,1,1,1], padding = "SAME")
    conv1 = tf.nn.relu(conv1)
    conv1_pool = tf.nn.max_pool(conv1, ksize =[1,2,2,1], strides = [1,2,2,1],padding= "SAME")
    conv1_bn = tf.layers.batch_normalization(conv1_pool)
    
    #3,4 
    conv2 = tf.nn.conv2d(conv1_bn, conv2_filter , strides = [1,1,1,1], padding = "SAME")
    conv2 = tf.nn.relu(conv2)
    conv2_pool = tf.nn.max_pool(conv2,ksize =[1,2,2,1],strides = [1,2,2,1],padding = "SAME")
    conv2_bn = tf.layers.batch_normalization(conv2_pool)
    
    #5,6 
    conv3 = tf.nn.conv2d(conv2_bn, conv3_filter, strides = [1,1,1,1],padding = "SAME")
    conv3 = tf.nn.relu(conv3)
    conv3_pool = tf.nn.max_pool(conv3,ksize = [1,2,2,1],strides = [1,2,2,1],padding = "SAME")
    conv3_bn = tf.layers.batch_normalization(conv3_pool)
    
    #7,8
    conv4 = tf.nn.conv2d(conv3_bn, conv4_filter, strides = [1,1,1,1], padding = "SAME")
    conv4 = tf.nn.relu(conv4)
    conv4_pool = tf.nn.max_pool(conv4,ksize = [1,2,2,1],strides = [1,2,2,1],padding = "SAME")
    conv4_bn = tf.layers.batch_normalization(conv4_pool)
    
    #fltten the input
    
    #9
    flat = tf.contrib.layers.flatten(conv4_bn)

    #fully connected layers
    
    #full1
    
    #10
    full1 = tf.contrib.layers.fully_connected(inputs = flat, num_outputs = 128 , activation_fn = tf.nn.relu)
    full1 = tf.nn.dropout(full1, keep_prob)
    full1 = tf.layers.batch_normalization(full1)
    
    #11
    full2 = tf.contrib.layers.fully_connected(inputs = full1, num_outputs = 256 , activation_fn = tf.nn.relu)
    full2 = tf.nn.dropout(full2, keep_prob)
    full2 = tf.layers.batch_normalization(full2)
    
    #12
    full3 = tf.contrib.layers.fully_connected(inputs = full2, num_outputs = 512 , activation_fn = tf.nn.relu)
    full3 = tf.nn.dropout(full3, keep_prob)
    full3 = tf.layers.batch_normalization(full3)
    
    #13
    full4 = tf.contrib.layers.fully_connected(inputs = full3, num_outputs = 1024 , activation_fn = tf.nn.relu)
    full4 = tf.nn.dropout(full4, keep_prob)
    full4 = tf.layers.batch_normalization(full4)
    
    #output layer
    
    out = tf.contrib.layers.fully_connected(inputs= full4 , num_outputs = 10, activation_fn = None)
    
    return out
    

Neural network is built

# Hyper parameters

In [31]:
epochs = 3
batch_size = 128
learning_rate = 0.001
keep_probability = 0.7

In [18]:
logits = conv_net(x, keep_prob)
model = tf.identity(logits, name = "logits") # name the logits tensor,so that it can be loaded after model is trained

In [19]:
# cost 

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = y))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [20]:
# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)


In [21]:
# accuracy

correct_pred = tf.equal(tf.argmax(logits , 1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred , tf.float32),name = 'accuracy')

In [22]:
# single optimization

def train_neural_network(session,optimizer,keep_probability,feature_batch, label_batch):
    session.run(optimizer,
                feed_dict = {x : feature_batch,
                y : label_batch,
                keep_prob : keep_probability})
    

In [24]:
# showing the stats

def print_stats(session, feature_batch , label_batch, cost , accuracy):
    loss = session.run(cost, feed_dict = {
        x : feature_batch,
        y : label_batch,
        keep_prob : 1
    })
    
    valid_accc = session.run(accuracy , feed_dict = {
        x: feature_batch,
        y : label_batch,
        keep_prob : 1
    })
    
    print('Loss : {:>10.4f}, Validation_accuracy:{:.6f}'.format(loss,valid_accc))

In [29]:
import pickle
def batch_features_labels(features,labels,batch_size):
    for start in range(0,len(features),batch_size):
        end = min(start + batch_size , len(features))
        yield features[start:end], labels[start:end]

def load_preprocess_training_batch(batch_id, batch_size):
    file_name = "preprocess_batch_" + str(batch_id) + ".p"
    features,labels = pickle.load(open(file_name,mode = 'rb'))
    
    return batch_features_labels(features,labels,batch_size)

# Running the tf model

In [32]:
save_model_path = './image_classification'
print("Training...............")

with tf.Session() as sess:
    
    # initializing the varibles
    sess.run(tf.global_variables_initializer())
    
    # training cycle:
    
    for epoch in range(epochs):
        no_batches =5
        
        for batch_i in range(1,no_batches):
            for batch_features, batch_labels in load_preprocess_training_batch(batch_i,batch_size):
                train_neural_network(sess,optimizer,keep_probability,batch_features, batch_labels)
            print('Epoch{:>2},Cifar 10 batch{}'.format(epoch+1,batch_i))
            print_stats(sess, batch_features , batch_labels, cost , accuracy)
            
        

Training...............
Epoch 1,Cifar 10 batch1
Loss :     2.3001, Validation_accuracy:0.125000
Epoch 1,Cifar 10 batch2
Loss :     2.2998, Validation_accuracy:0.175000
Epoch 1,Cifar 10 batch3
Loss :     2.2931, Validation_accuracy:0.200000


KeyboardInterrupt: 